# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

Calculation started (calculation_id=1ac7128f-1bce-8eca-eb06-674abceeb62e) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=00c7128f-215f-5672-8004-de0cec51bec2) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=cac7128f-22f4-94a4-acc4-df9503c33979) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [4]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'xjiao34'

Calculation started (calculation_id=4cc7128f-2410-d7c1-dd51-881165e06cb8) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:

    # filter the result
    trips_filtered = trips.filter(col("trip_distance") >= 2)
    return trips_filtered

Calculation started (calculation_id=26c7128f-2547-c935-247c-cdfd666709e6) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [6]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    # join
    # filter
    trip_lookup = trips.join(lookup,col("DOLocationID") == col("LocationID")).filter(col("Borough") == "Manhattan")
    
    # group by
    # agg
    # rename
    dfc = trip_lookup.groupBy(col("DOLocationID")).agg({"passenger_count":"sum"}).withColumn('pcount',col("sum(passenger_count)"))
       
    # select
    # order by
    # limit
    manhattan_trip_df = dfc.select(["DOLocationID","pcount"]).orderBy(col("pcount"),ascending =False).limit(20)
    
    return manhattan_trip_df

Calculation started (calculation_id=c2c7128f-2674-a3f4-dff0-43d190f22e06) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [20]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    

    # Alias the trips DataFrame for clarity
    df_trips = trips.alias("trips")

    # Calculate the average total amount and count of trips for each PULocationID
    df_avg = df_trips.withColumn("total_amount", col("total_amount").cast("double")) \
                     .groupBy("trips.PULocationID") \
                     .agg(avg("total_amount").alias("avg_d"), count("trips.PULocationID").alias("pucount")) \
                     .select(col("trips.PULocationID").alias("PULocationID"), col("avg_d"), col("pucount"))

    # Prepare the mtrips DataFrame for joining and apply alias
    df_mtrips = mtrips.alias("mtrips")
    dt = df_mtrips.join(df_trips, col("mtrips.DOLocationID") == col("trips.DOLocationID"), "left") \
                  .groupBy("trips.PULocationID") \
                  .agg(count("trips.trip_distance").alias("p_count")) \
                  .select(col("trips.PULocationID").alias("PULocationID"), col("p_count"))

    # Join dt and df_avg DataFrames and calculate weighted_profit
    # Make sure to use the column names directly without alias prefixes as they are already aliased
    df_final = dt.join(df_avg, dt["PULocationID"] == df_avg["PULocationID"], "left") \
           .withColumn("weighted_profit", col("p_count") / col("pucount") * col("avg_d")) \
           .withColumn("weighted_profit", col("weighted_profit").cast("double")) \
           .select(dt["PULocationID"], col("weighted_profit"))

    return df_final

Calculation started (calculation_id=40c71293-3c7c-55a8-4ae2-06c37d79a658) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [8]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    # join
    # select
    df1 = wp.join(lookup, wp.PULocationID == lookup.LocationID).select("Zone","Borough","weighted_profit")
    # order by
    # limit
    df_final5 = df1.orderBy("weighted_profit", ascending=False).limit(20)
    return df_final5
    

Calculation started (calculation_id=30c7128f-2a91-41bd-fbec-575f1ad290b0) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [22]:
bucket = 's3://cse6242-xjiao34/output_large'

main('large', bucket)

Calculation started (calculation_id=50c71293-b6f8-9eee-d9ae-2b2bf9a6e02d) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
xjiao34
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens| 29.36045577913084|
|Flushing Meadows-...|       Queens|27.304845733617675|
|       South Jamaica|       Queens| 26.29491623987346|
|     Randalls Island|    Manhattan| 24.15098994022752|
|        Astoria Park|       Queens|21.706417112147523|
|Briarwood/Jamaica...|       Queens|19.945064631789336|
|Springfield Garde...|       Queens|19.468309288781906|
|             Jamaica|       Queens|  19.2839430001379|
|              Corona|       Queens|18.228769248155974|
|   LaGuardia Airport|       Queens|18.181338808372995|
|         Jamaica Bay|       Queens|17.100529446757896|
|             Maspeth|       Queens| 17.00545064007954|
|Eltingville/Annad...|Staten Island|16.837764756944445|
|         JFK Airport|       Queens|16.77772534824

In [9]:
df = load_data(size='small')

df[0].printSchema()  
df[1].printSchema()

Calculation started (calculation_id=04c7128f-3217-adb9-1b03-69ea0260e420) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Trip Count:  7667792
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- s

In [10]:
trips = df[0]
lookup = df[1]

trips = long_trips(trips)

trips.show(5)

Calculation started (calculation_id=54c7128f-708a-bbec-5f3e-fde84c504dbf) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:59:47|  2019-01-01 01:18:59|              1|          2.6|         1|                 N|         239|         246|           1|       14.0|  0.5|    0.5|       1.0|         0.

In [11]:
df4 = manhattan_trips(trips, lookup)

df4.show(20)

Calculation started (calculation_id=7ec7128f-75bd-e547-2cb9-4f073541dffb) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------------+------+
|DOLocationID|pcount|
+------------+------+
|         236|147086|
|         161|117642|
|         230|112155|
|         239|104164|
|         162|101209|
|         238|100189|
|          48| 98784|
|         170| 96109|
|         142| 95053|
|         231| 94621|
|         263| 94548|
|          79| 94421|
|         237| 90335|
|         141| 90061|
|         140| 84212|
|          68| 79559|
|         234| 75786|
|         186| 73863|
|         163| 73145|
|          13| 70693|
+------------+------+



In [21]:
mtrips = df4
df5 = weighted_profit(trips, mtrips)

df5.show()

Calculation started (calculation_id=76c71293-4c95-1be0-53db-eb952eda2717) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------------+------------------+
|PULocationID|   weighted_profit|
+------------+------------------+
|           1| 4.242351020408163|
|          12|10.554036603239659|
|          37|2.3007539038979696|
|          61| 2.850152482392189|
|          65| 6.630698230351476|
|          81| 4.005536332179929|
|          92|3.5754301901107586|
|         107|10.677769405056726|
|         112| 3.809072830499689|
|         126|  5.50219538703055|
|         140|  8.28884629143913|
|         142|7.9056723642971365|
|         148| 7.697406151366665|
|         178| 4.461430078727375|
|         189|3.8735747720721547|
|         192| 3.407708576894462|
|         205|  6.96914891929408|
|         209| 9.291515922789104|
|         243|  7.93404250624856|
|         250| 4.283196681182198|
+------------+------------------+
only showing top 20 rows



In [19]:
df6 = final_output(df5, lookup)

df6.show()

Calculation started (calculation_id=9ec71292-af87-ed03-7543-208671df187a) in (session=82c7124e-d84b-5740-ce1a-a75857c052f0). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|Arrochar/Fort Wad...|Staten Island|31.784444444444446|
|     Freshkills Park|Staten Island|29.342499999999998|
|       Port Richmond|Staten Island|24.357777777777777|
|Flushing Meadows-...|       Queens| 24.33013773166695|
|        Baisley Park|       Queens| 23.62469433349846|
|     Randalls Island|    Manhattan|23.124577515149234|
|       South Jamaica|       Queens| 22.17411493943223|
|     Mariners Harbor|Staten Island|21.786556927297667|
|Saint George/New ...|Staten Island|21.491893491124262|
|       Arden Heights|Staten Island|          20.29625|
|        Astoria Park|       Queens|19.965360000000004|
|   LaGuardia Airport|       Queens| 17.92315394960938|
|  Murray Hill-Queens|       Queens|17.585450424382714|
|         JFK Airport|       Queens|16.958978201178994|
|             Jamaica|   